In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from architecture.deep_binary_classifier import DeepBinaryClassifier
from architecture.ripper_node import make_ripper_node
from architecture.lut_node import make_lut_node


df   = pd.read_csv("./data/100_bit_artificial/1a.csv")
X = df.drop(columns="class").to_numpy(bool)
y = df["class"].to_numpy(bool)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

config = dict(layer_node_counts=[32]*5 + [1], layer_bit_counts=[6]*6, seed=42)

In [2]:
# the LUT network runs faster on a single thread

lut_net = DeepBinaryClassifier(**config, node_factory=make_lut_node, jobs=1)
%time lut_net.fit(X_train, y_train)
pred_test = lut_net.predict(X_test)
acc_lut = accuracy_score(y_test, pred_test)
print(f"LUT network accuracy: {acc_lut:.4f}")

CPU times: user 41.1 ms, sys: 2.26 ms, total: 43.3 ms
Wall time: 42 ms
LUT network accuracy: 0.7355


In [3]:
# the Ripper nodes profit from parallelization

rip_net = DeepBinaryClassifier(**config, node_factory=make_ripper_node, jobs=8)
%time rip_net.fit(X_train, y_train)
pred_test = rip_net.predict(X_test)
acc_rip = accuracy_score(y_test, pred_test)
print(f"Rule network accuracy: {acc_rip:.4f}")

CPU times: user 160 ms, sys: 229 ms, total: 389 ms
Wall time: 28.8 s
Rule network accuracy: 0.8830


In [4]:
rip_node = rip_net.layers[0][5]
rip_node.ripper.out_model()

[[x_73=False ^ x_84=True ^ x_79=False]]


In [7]:
# rip_node is your RipperNode instance
r = rip_node.ripper.ruleset_

print(">> Ripper dir:")
print(dir(rip_node.ripper))
print()

print(">> Ruleset dir:")
print(dir(r))
print()

if len(r) > 0:
    print(">> First rule dir:")
    print(dir(r[0]))


>> Ripper dir:
['VALID_HYPERPARAMETERS', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', '_cover_remaining_positives', '_cover_remaining_positives_cn', '_ensure_has_bin_transformer', '_estimator_type', '_grow_ruleset', '_grow_ruleset_cn', '_optimize_ruleset', '_optimize_ruleset_cn', '_ruleset_frommodel', '_set_deprecated_fit_params', '_set_theory_dl_lookup', 'add_rule', 'algorithm_name', 'alpha', 'bin_transformer_', 'class_feat', 'classes_', 'copy', 'dl_allowance', 'fit', 'get_params', 'init_ruleset', 'insert_rule', 'insert_rule_at', 'k', 'max_rule_conds', 'max_rules', 'max_total_conds', 'n_discretize_bins', 'out_model', 'pos_class', 'predict', 'p

In [8]:
rip_node.ripper.selected_features_

['x5', 'x2', 'x4', 'x0', 'x3', 'x1']

In [5]:
# get al lthe col indices
rip_node.X_cols

NameError: name 'rip_node' is not defined

In [29]:
import numpy as np

# rip_node is your RipperNode instance
ripper = rip_node.ripper

print("RULES:")
for i, rule in enumerate(ripper.ruleset_):
    # rule.class_ns_ == [#covered class 0, #covered class 1]
    cls = int(np.argmax(rule.class_ns_))
    print(f"RULE {i}: IF {rule} THEN y={cls}")

# ➡️ Use smoothed_uncovered_class_freqs_ for the real fallback
smoothed = ripper.ruleset_.smoothed_uncovered_class_freqs_
default_cls = int(np.argmax(smoothed))
print(f"DEFAULT: y={default_cls}")

RULES:
RULE 0: IF [bit5=False^bit2=False^bit4=False^bit0=False^bit3=False^bit1=False] THEN y=1
DEFAULT: y=1


In [36]:
rip_node.ripper.ruleset_.uncovered_class_ns_

(3354, 4513)

In [23]:
import numpy as np
import pandas as pd
from architecture.utils import truth_table_patterns, truth_table_indices

# 1) Unpack
node   = rip_node
ripper = node.ripper
pred_node = node.pred_node

# 2) Determine bit-width and full input dim
n_bits = int(np.log2(pred_node.size))
D      = int(node.X_cols.max()) + 1

# 3) Generate all bit patterns for the node’s inputs
patterns = truth_table_patterns(n_bits)             # shape (2**n_bits, n_bits)

# 4) Build full-width X so node.__call__ works
X_full = np.zeros((len(patterns), D), dtype=bool)
X_full[:, node.X_cols] = patterns

# 5) Predict via precomputed lookup
pred_via_node = node(X_full)                        # uses pred_node internally

# 6) Predict via live RIPPER.predict
names = [f"bit{i}" for i in range(n_bits)]
df = pd.DataFrame(patterns, columns=names).astype(bool)
raw_preds = ripper.predict(df)
pred_via_predict = np.asarray(raw_preds, dtype=bool)

# 7) Compare
equal = np.array_equal(pred_via_predict, pred_via_node)
print("Consistency between ripper.predict and pred_node:", equal)
if not equal:
    diffs = np.where(pred_via_predict != pred_via_node)[0]
    print("First few mismatches:")
    for i in diffs[:5]:
        print(f" idx {i}: pattern={patterns[i]}, predict={pred_via_predict[i]}, lookup={pred_via_node[i]}")


Consistency between ripper.predict and pred_node: True


In [26]:
import numpy as np
import pandas as pd
from architecture.utils import truth_table_patterns

# rip_node is your RipperNode instance
ripper    = rip_node.ripper
pred_node = rip_node.pred_node

# 1) regenerate all patterns
n_bits   = int(np.log2(pred_node.size))
patterns = truth_table_patterns(n_bits)  # shape (2**n_bits, n_bits)

print("Checking each rule against pred_node…")
for i, rule in enumerate(ripper.ruleset_):
    # predict‐by‐counts
    cls = int(np.argmax(rule.class_ns_))

    # parse "[bit5=False^bit2=False…]" → list of "bitX=Y"
    conds = str(rule).strip("[]").split("^")

    # build mask: start all‐True, AND each condition
    mask = np.ones(len(patterns), dtype=bool)
    for clause in conds:
        name, val = clause.split("=")
        bit = int(name.replace("bit", ""))
        mask &= (patterns[:, bit] == (val == "True"))

    # now verify pred_node[mask] == cls
    if not np.all(pred_node[mask] == cls):
        bad = np.where(pred_node[mask] != cls)[0]
        print(f"❌ Rule {i} mismatch at local indices {bad[:5]} (predicted {cls})")
    else:
        print(f"✅ Rule {i} OK (predicts {cls}, covers {mask.sum()} patterns)")

# 2) check default: uncovered patterns
covered = np.zeros(len(patterns), dtype=bool)
for rule in ripper.ruleset_:
    conds = str(rule).strip("[]").split("^")
    tmp = np.ones(len(patterns), dtype=bool)
    for clause in conds:
        name, val = clause.split("=")
        bit = int(name.replace("bit", ""))
        tmp &= (patterns[:, bit] == (val == "True"))
    covered |= tmp

uncovered = ~covered
default = int(np.argmax(ripper.ruleset_.smoothed_uncovered_class_freqs_))
if not np.all(pred_node[uncovered] == default):
    bad = np.where(pred_node[uncovered] != default)[0]
    print(f"❌ Default mismatch at indices {bad[:5]}")
else:
    print(f"✅ Default OK (predicts {default}, covers {uncovered.sum()} patterns)")


Checking each rule against pred_node…
✅ Rule 0 OK (predicts 1, covers 1 patterns)
❌ Default mismatch at indices [0 1 2 3 4]


In [27]:
import numpy as np
import pandas as pd
from architecture.utils import truth_table_patterns

# 1) Grab your objects
node      = rip_node
ripper    = node.ripper
pred_node = node.pred_node

# 2) Figure out how many bits this node handles
n_bits = int(np.log2(pred_node.size))

# 3) Create the full truth table
patterns = truth_table_patterns(n_bits)        # shape (2**n_bits, n_bits)
names    = [f"bit{i}" for i in range(n_bits)]
df       = pd.DataFrame(patterns, columns=names).astype(bool)

# 4) Ask RIPPER directly
raw_preds       = ripper.predict(df)
pred_via_predict = np.asarray(raw_preds, dtype=bool)

# 5) Compare to your stored lookup
ok = np.array_equal(pred_via_predict, pred_node)
print("Ruleset ↔ pred_node consistent?", ok)

if not ok:
    diff = np.where(pred_via_predict != pred_node)[0]
    print("First mismatches at indices:", diff[:5])
    for i in diff[:5]:
        print(f" pattern {patterns[i]} → ripper.predict: {pred_via_predict[i]}, lookup: {pred_node[i]}")


Ruleset ↔ pred_node consistent? True
